###  Module & Utility Imports

In [1]:
import pandas as pd
import numpy as np

### Iscore: Comparison between the main model and benchmark models

In [2]:
# Re-load the original dataframe
BenchAnalAccMItable = pd.read_csv('./EvalResults/SummaryTables/BenchAnalAccMItable.csv')

# Pivot table with 'ISCOREScal' values
PivotTable = BenchAnalAccMItable.pivot_table(index=['Source', 'Type'], columns='Prefix', values='ISCOREScal').reset_index()

# Rename 'SKZFC' column to 'Ours'
PivotTable = PivotTable.rename(columns={'SKZFC': 'Ours'})

# Move 'Ours' column to the end
cols = [col for col in PivotTable.columns if col != 'Ours'] + ['Ours']
PivotTable = PivotTable[cols]

# Generate LaTeX table code with \begin{table}...\end{table} structure
latex_table_code = r"""\begin{table}[t]
\centering
\caption{Comparison of ISCOREScal values across models, grouped by Source and Type.}
\label{tab:iscore_comparison}
""" + PivotTable.to_latex(index=False, float_format="%.4f", column_format="ll" + "r"*(len(PivotTable.columns)-2), escape=False, bold_rows=False) + r"""\end{table}"""

# Print the LaTeX table code
print(latex_table_code)


\begin{table}[t]
\centering
\caption{Comparison of ISCOREScal values across models, grouped by Source and Type.}
\label{tab:iscore_comparison}
\begin{tabular}{llrrrrrrrr}
\toprule
Source & Type & ConVAE & DiffWave & FACVAE & TCVAE & VDVAE & VDWave & Wavenet & Ours \\
\midrule
Mimic & ART & 0.2027 & 0.3337 & 0.1963 & 0.1199 & 0.1303 & 0.3338 & 0.2356 & 2.0832 \\
Mimic & II & 0.5653 & 0.6366 & 0.2435 & 0.5838 & 0.2548 & 0.6374 & 0.2583 & 2.4422 \\
VitalDB & ART & 0.1528 & 0.3419 & 0.1604 & 0.1867 & 0.1369 & 0.3420 & 0.1804 & 1.9574 \\
VitalDB & II & 0.4400 & 0.6585 & 0.4000 & 0.5647 & 0.3364 & 0.6590 & 0.4718 & 2.0179 \\
\bottomrule
\end{tabular}
\end{table}


### Evaluation metrics: Comparison between the main model and benchmark models

In [6]:
# Check the actual column names
actual_columns = BenchAnalAccMItable.columns.tolist()

# Define major metrics based on actual column names
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]

# Get unique values for Source and Type columns
sources = BenchAnalAccMItable['Source'].unique()
types = BenchAnalAccMItable['Type'].unique()

# Rename 'SKZFC' to 'Ours' for easier interpretation
BenchAnalAccMItable.loc[BenchAnalAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'

# Initialize a dictionary to store the generated LaTeX tables
split_tables_latex = {}

# Generate LaTeX table for each (Source, Type) pair
for source in sources:
    for type_ in types:
        # Filter by Source and Type
        subset = BenchAnalAccMItable[(BenchAnalAccMItable['Source'] == source) & (BenchAnalAccMItable['Type'] == type_)]
        
        # Pivot to organize metrics by model prefix
        table = subset.pivot_table(index='Prefix', values=major_metrics)
        table = table.reset_index()
        
        # Reorder columns: Model name first, then major metrics
        table = table[['Prefix'] + major_metrics].copy()
        
        # Rename columns
        table = table.rename(columns={
            'Prefix': 'Model', 
            '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
            '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
            '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$', 
            'FQI': 'FQI $\\downarrow$',
            'RMSE': 'RMSE $\\downarrow$',
            'ISCOREScal': 'ISCORE $\\uparrow$'
        })
        
        # Convert table to LaTeX with custom structure
        latex_code = r"""\begin{table}[t]
\centering
\caption{Results for Source=""" + f"{source}" + r""", Type=""" + f"{type_}" + r"""}
\label{tab:result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format="l" + "r"*(len(table.columns)-1), float_format="%.4f") + r"""\end{table}"""

        # Save the LaTeX code
        split_tables_latex[(source, type_)] = latex_code

# Example: print one latex table
print(split_tables_latex[('Mimic', 'ART')])


\begin{table}[t]
\centering
\caption{Results for Source=Mimic, Type=ART}
\label{tab:result_mimic_art}
\begin{tabular}{lrrrrrr}
\toprule
Model & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
ConVAE & 0.0519 & 0.9339 & 0.2500 & 0.0980 & 3.1147 & 0.2027 \\
DiffWave & 0.0000 & 0.0018 & 0.0005 & 0.0439 & 5.0266 & 0.3337 \\
FACVAE & 0.0609 & 0.9601 & 0.2057 & 0.0885 & 2.9360 & 0.1963 \\
Ours & 3.6300 & 0.0478 & 0.8360 & 0.0883 & 3.6893 & 2.0832 \\
TCVAE & 0.1286 & 1.5965 & 0.2364 & 0.1085 & 3.2440 & 0.1199 \\
VDVAE & 0.0655 & 1.3576 & 0.1124 & 0.0890 & 2.8555 & 0.1303 \\
VDWave & 0.0000 & 0.0014 & 0.0004 & 0.0437 & 5.0093 & 0.3338 \\
Wavenet & 0.0000 & 0.4754 & 0.0487 & 0.0747 & 4.6166 & 0.2356 \\
\bottomrule
\end{tabular}
\end{table}


In [ ]:
print(split_tables_latex[('VitalDB', 'II')])